In [ ]:
! git clone https://github.com/hossamhamdy333/AI-Projects-Portfolio.git


Cloning into 'AI-Projects-Portfolio'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 81 (delta 10), reused 56 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 2.57 MiB | 17.28 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
print("hello world")

hello world


In [3]:
! ls


AI-Projects-Portfolio  sample_data


In [23]:
!cd medical_cost_prediction

/bin/bash: line 1: cd: medical_cost_prediction: No such file or directory


In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [28]:
!ls "/content/drive/MyDrive/"


'can you give me more details.gsheet'
 Classroom
'Colab Notebooks'
 cs231n
'hossam hamdy 20210369.pdf'
'IMG_20250215_035727 (1).jpg'
 IMG_20250215_035727.jpg
'IMG_20250215_035731 (1).jpg'
 IMG_20250215_035731.jpg
 inbound6114845725863208215.jpg
'Jam بلا عنوان.pdf'
 project1
 rec_o2022-9.gsheet
'Untitled spreadsheet.gsheet'
'WhatsApp Image 2023-07-03 at 02.53.40.jpg'
'WhatsApp Image 2023-07-03 at 12.15.39.jpg'
'WhatsApp Image 2023-07-05 at 02.15.56.jpg'
'فاعل خير'
'ملزمة متطلبات الفروع.gdoc'


In [24]:
!cp "/content/drive/MyDrive/colab Notebomedical_cost_prediction.ipynb" .


cp: cannot stat '/content/drive/MyDrive/medical_cost_prediction.ipynb': No such file or directory
